# Module 01 — Nodes, Topics & Messages

## How to use this module

This module is a **lab session**, not a lecture.

Every section follows the same pattern:
1. You observe something
2. You make a prediction
3. You run an experiment
4. You draw a conclusion

The explanations come **after** the experiments — never before.

---

## 🎥 Watch First

<div style="padding:56.25% 0 0 0;position:relative;">
  <iframe src="https://player.vimeo.com/video/YOUR_VIDEO_ID"
          style="position:absolute;top:0;left:0;width:100%;height:100%;"
          frameborder="0" allow="autoplay; fullscreen; picture-in-picture" allowfullscreen>
  </iframe>
</div>

---

## Part 1 — What is a Node?

### 🧪 Experiment 1.1 — Start the robot and observe

Open your desktop and run in Terminator:

```bash
python3 /home/jovyan/course_materials/scripts/robot_simulator.py
```

Now in a **second terminal** run:

```bash
ros2 node list
```

**What do you see?**

> 📝 Write your observation here before reading on.

---

### 🔭 Observation

You should see:
```
/mobile_robot
```

One entry. One **node**. The simulator is running as a single ROS2 process with the name `/mobile_robot`.

### 🧪 Experiment 1.2 — Get more information

```bash
ros2 node info /mobile_robot
```

**Before running:** What do you expect to see?

> 📝 Write your prediction here.

**After running:** Were you right?

You'll see the node's **publishers**, **subscribers**, **services**, and **parameters**. This is the complete interface of the robot — everything it can send, receive, and configure.

---

### 💡 Conclusion: What is a Node?

> A **Node** is an independent process that does one job in a robot system.

Think of a robot system as a team. Each team member (node) has a specific role:
- Camera node — captures images
- Motor controller node — drives the wheels
- Planning node — decides where to go
- Sensor fusion node — combines sensor data

Each node runs independently. If one crashes, the others keep running.

---

## Part 2 — What is a Topic?

### 🧪 Experiment 2.1 — Discover the robot's channels

```bash
ros2 topic list
```

You should see several topics. Focus on these:
```
/cmd_vel
/odom
/battery_state
```

**Before the next step:** Look at the names. `/cmd_vel`, `/odom`, `/battery_state`.

What do you think each one contains? Write your hypothesis:

> `/cmd_vel` — I think this contains: ___

> `/odom` — I think this contains: ___

> `/battery_state` — I think this contains: ___

---

### 🧪 Experiment 2.2 — Look inside a topic

```bash
ros2 topic echo /odom
```

Let it run for 3 seconds, then Ctrl+C.

**What do you see?** Is it what you predicted?

Now check the battery:
```bash
ros2 topic echo /battery_state
```

Watch the `percentage` field. **What is it doing?**

---

### 🧪 Experiment 2.3 — How fast is data flowing?

```bash
ros2 topic hz /odom
```

**What does this number mean?** 

Now check `/battery_state`:
```bash
ros2 topic hz /battery_state
```

Are they the same frequency? Why might they be different?

---

### 🧪 Experiment 2.4 — The most important topic: /cmd_vel

Keep the simulator running. In a second terminal:

```bash
ros2 topic echo /cmd_vel
```

Notice: **nothing is printing.** Why?

> 📝 Write your hypothesis.

Now in a **third terminal**, send a command:
```bash
ros2 topic pub /cmd_vel geometry_msgs/msg/Twist "{linear: {x: 0.5}}" --once
```

Watch the `/cmd_vel` echo terminal. **What appeared?**

Watch the `/odom` terminal. **What changed?**

---

### 💡 Conclusion: What is a Topic?

> A **Topic** is a named channel through which nodes exchange data continuously.

Key properties you just discovered experimentally:
- Topics exist even when nobody is publishing to them
- Multiple terminals can listen to the same topic simultaneously
- Data flows in one direction — publishers send, subscribers receive
- Publishers don't know who is listening, subscribers don't know who is sending

This independence is what makes ROS2 powerful — you can add a new node that listens to `/odom` without touching the robot simulator at all.

---

## Part 3 — What is a Message?

### 🧪 Experiment 3.1 — Inspect the data type

```bash
ros2 topic info /cmd_vel
```

You'll see: `Type: geometry_msgs/msg/Twist`

What is a `Twist`? Let's find out:

```bash
ros2 interface show geometry_msgs/msg/Twist
```

**Read the output carefully.** You'll see:
```
Vector3 linear
    float64 x
    float64 y
    float64 z
Vector3 angular
    float64 x
    float64 y
    float64 z
```

---

### 🧪 Experiment 3.2 — Explore the fields

Earlier you sent:
```bash
ros2 topic pub /cmd_vel geometry_msgs/msg/Twist "{linear: {x: 0.5}}" --once
```

The robot moved **forward**. `linear.x` = forward/backward speed.

**Predict what each of these will do before running:**

> `linear.x: -0.5` — I predict: ___

> `angular.z: 1.0` — I predict: ___

> `linear.x: 0.3, angular.z: 1.0` — I predict: ___

Now test each prediction:

```bash
# Test 1
ros2 topic pub /cmd_vel geometry_msgs/msg/Twist "{linear: {x: -0.5}}" --once

# Test 2
ros2 topic pub /cmd_vel geometry_msgs/msg/Twist "{angular: {z: 1.0}}" --once

# Test 3
ros2 topic pub /cmd_vel geometry_msgs/msg/Twist "{linear: {x: 0.3}, angular: {z: 1.0}}" --once
```

**How many predictions were correct?**

---

### 🧪 Experiment 3.3 — Continuous vs one-shot publishing

Remove the `--once` flag:
```bash
ros2 topic pub /cmd_vel geometry_msgs/msg/Twist "{linear: {x: 0.3}, angular: {z: 0.5}}"
```

Watch the robot in RViz. **What's different from before?**

Now press **Ctrl+C** to stop publishing.

**What happens to the robot?** Does it stop immediately or keep moving? Why?

---

### 💡 Conclusion: What is a Message?

> A **Message** is a typed data structure exchanged over a topic.

From your experiments you discovered:
- `geometry_msgs/msg/Twist` has `linear` and `angular` components
- `linear.x` controls forward/backward speed (m/s)
- `angular.z` controls rotation speed (rad/s)
- These exact same message types are used on every mobile robot in ROS2

Common message types you'll use in this course:

| Message | Used for |
|---------|----------|
| `geometry_msgs/Twist` | Velocity commands |
| `nav_msgs/Odometry` | Robot position and speed |
| `sensor_msgs/LaserScan` | LiDAR data |
| `sensor_msgs/Image` | Camera images |
| `geometry_msgs/PoseStamped` | Navigation goals |

---

## Part 4 — Visualize the System

### 🧪 Experiment 4.1 — rqt_graph

With the simulator still running, in a new terminal:

```bash
rqt_graph
```

You'll see the node graph of your system.

**Answer these questions by looking at the graph:**

1. How many nodes are running?
2. Which topics connect them?
3. What is producing `/odom`? What is consuming it?

---

### 🧪 Experiment 4.2 — Add your own node to the graph

In a new terminal, start listening to `/odom`:
```bash
ros2 topic echo /odom > /dev/null &
```

Now refresh rqt_graph. **What changed?**

You added a subscriber to the graph without touching the simulator at all. This is the publish-subscribe model in action.

Kill the background process when done:
```bash
kill %1
```

---

## Part 5 — Set Up Your Workspace

You've been controlling the robot from the command line. Now let's write Python code that does the same thing — a node that publishes `/cmd_vel` automatically.

### Verify ROS2 is ready


In [ ]:
import subprocess

result = subprocess.run(['ros2', '--version'], capture_output=True, text=True)
print(result.stdout)
print('ROS2 is ready!' if result.returncode == 0 else 'ROS2 not found')


### Create your workspace

Run these in Terminator:


In [ ]:
%%bash
mkdir -p ~/ros2_ws/src
cd ~/ros2_ws
colcon build
echo 'Workspace ready!'


### Create your first package


In [ ]:
%%bash
cd ~/ros2_ws/src
ros2 pkg create --build-type ament_python ros2_tutorials
echo 'Package created!'


### Workspace structure

```
ros2_ws/
└── src/
    └── ros2_tutorials/
        ├── ros2_tutorials/   ← your Python nodes go here
        │   └── __init__.py
        ├── package.xml
        └── setup.py          ← register your nodes here
```

---

## Part 6 — Write Your First Node

### 🧪 Experiment 6.1 — Read the simulator source

Before writing your own node, read how the simulator publishes `/odom`:

```bash
cat /home/jovyan/course_materials/scripts/robot_simulator.py
```

Find these lines and understand them:
```python
self.odom_pub = self.create_publisher(Odometry, 'odom', 10)
self.timer = self.create_timer(1.0 / rate, self.update)
```

**Before continuing:** What does `10` mean in `create_publisher`? What does `1.0 / rate` control?

> 📝 Write your hypothesis.

---

### Write a velocity publisher

Now write a node that does what you were doing manually — publishing to `/cmd_vel`:


In [ ]:
%%writefile ~/ros2_ws/src/ros2_tutorials/ros2_tutorials/velocity_publisher.py
#!/usr/bin/env python3
import rclpy
from rclpy.node import Node
from geometry_msgs.msg import Twist

class VelocityPublisher(Node):
    def __init__(self):
        super().__init__('velocity_publisher')
        self.publisher_ = self.create_publisher(Twist, 'cmd_vel', 10)
        self.timer = self.create_timer(0.5, self.timer_callback)
        self.get_logger().info('Velocity publisher started!')

    def timer_callback(self):
        msg = Twist()
        msg.linear.x = 0.3
        msg.angular.z = 0.5
        self.publisher_.publish(msg)
        self.get_logger().info(f'Publishing: x={msg.linear.x}, z={msg.angular.z}')

def main(args=None):
    rclpy.init(args=args)
    node = VelocityPublisher()
    rclpy.spin(node)
    node.destroy_node()
    rclpy.shutdown()

if __name__ == '__main__':
    main()


### Register the node in setup.py

Edit `~/ros2_ws/src/ros2_tutorials/setup.py` and add to `entry_points`:

```python
entry_points={
    'console_scripts': [
        'py_velocity_publisher = ros2_tutorials.velocity_publisher:main',
    ],
},
```

### Build and run


In [ ]:
%%bash
cd ~/ros2_ws
colcon build
source install/setup.bash
echo 'Build complete!'


With the simulator running, in a new terminal:

```bash
cd ~/ros2_ws && source install/setup.bash
ros2 run ros2_tutorials py_velocity_publisher
```

**Watch the robot in RViz.** It should be driving in a circle automatically.

Now open rqt_graph. **How many nodes do you see now? What connects them?**

---

## Part 7 — Write a Subscriber

### 🧪 Experiment 7.1 — Predict the structure

You just wrote a publisher. A subscriber is the mirror image.

**Before looking at the code below**, try to predict the key differences:

> Instead of `create_publisher`, you'll use: ___

> Instead of a timer calling a function, you'll use: ___

Now write it and see if you were right:


In [ ]:
%%writefile ~/ros2_ws/src/ros2_tutorials/ros2_tutorials/odom_subscriber.py
#!/usr/bin/env python3
import rclpy
from rclpy.node import Node
from nav_msgs.msg import Odometry

class OdomSubscriber(Node):
    def __init__(self):
        super().__init__('odom_subscriber')
        self.subscription = self.create_subscription(
            Odometry, 'odom', self.odom_callback, 10)
        self.get_logger().info('Listening to /odom...')

    def odom_callback(self, msg):
        x = msg.pose.pose.position.x
        y = msg.pose.pose.position.y
        vx = msg.twist.twist.linear.x
        self.get_logger().info(f'Position: x={x:.2f}, y={y:.2f} | Speed: {vx:.2f} m/s')

def main(args=None):
    rclpy.init(args=args)
    node = OdomSubscriber()
    rclpy.spin(node)
    node.destroy_node()
    rclpy.shutdown()

if __name__ == '__main__':
    main()


Add to `setup.py` entry_points:
```python
'py_odom_subscriber = ros2_tutorials.odom_subscriber:main',
```


In [ ]:
%%bash
cd ~/ros2_ws && colcon build && source install/setup.bash
echo 'Done!'


Run with the simulator and velocity publisher both active:

```bash
ros2 run ros2_tutorials py_odom_subscriber
```

You now have 3 nodes running simultaneously:
- `/mobile_robot` — the robot simulator
- `/velocity_publisher` — sending commands
- `/odom_subscriber` — reading position

**Open rqt_graph. Draw what you see on paper.** This is a real robot control loop.

---

## 🧠 Conceptual Challenges

These questions have no code answers. Think about the system you built.

**1.** You have a camera node publishing images and a display node showing them. If you add a **second** display node, does the camera node need to change anything? Why?

**2.** Your velocity publisher is running and the robot is moving. You kill the publisher with Ctrl+C. What happens to the robot? What would happen on a real physical robot?

**3.** You run `ros2 topic echo /odom` and see the position updating. You run it a second time in another terminal. Does the first terminal slow down? Why or why not?

**4.** Look at your rqt_graph with all 3 nodes running. The simulator crashes. Draw the new graph. What can still function?

**5.** `/cmd_vel` uses `geometry_msgs/msg/Twist`. A TurtleBot4, an Andino, and your simulator all use the same message type. What does this mean for code you write — can you reuse it across robots?

---

## 📚 Summary

From your experiments today you discovered:

✅ A **Node** is an independent process — each does one job

✅ A **Topic** is a named channel — nodes communicate without knowing each other

✅ A **Message** is a typed data structure — `Twist`, `Odometry`, `LaserScan`

✅ Publishers and subscribers are completely **decoupled** — add or remove either without touching the other

✅ `rqt_graph` shows you the live architecture of any ROS2 system

➡️ **Next:** Module 02 — Services & Parameters